In [2]:
import numpy as np
from numpy import linalg as LA  # Notar que importamos linalg con nickname LA

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [3]:
c1 = 2*(np.random.rand(3,1)-0.5)
print(c1)
c1.shape

[[ 0.54460658]
 [ 0.12336894]
 [-0.1551485 ]]


(3, 1)

In [4]:
"""Queremos entender el proceso de Gram-Schmidt."""

# Vamos a generar tres vectores al azar
# las chances son que van a ser linealmente independientes (Por que?)

c1 = 2*(np.random.rand(3,1)-0.5) # c1, c2 y c3 son dos vectores 3x1 con elementos aleatorios en el 
c2 = 2*(np.random.rand(3,1)-0.5) # rango (-1,1). Pregunta: por que puedo sumar un vector                             
c3 = 2*(np.random.rand(3,1)-0.5) # y un escalar? Buscar "numpy broadcasting"

print("c1 =", c1)
print("c2 =", c2)
print("c3 =", c3)

c1 = [[ 0.16144846]
 [-0.99824578]
 [ 0.05610655]]
c2 = [[ 0.48279885]
 [-0.09739081]
 [-0.6289578 ]]
c3 = [[ 0.95948716]
 [-0.67047217]
 [-0.89067367]]


In [5]:
# si queremos chequear que no son linealmente independientes

print(np.dot(np.transpose(c1),c2)/(LA.norm(c1)*LA.norm(c2)))
# EL producto escalar entre c1 y c2 es c1.c2 = |c1||c2|cos(theta), entonces, 
# c1.c2/(|c1||c2|) = cos(theta), por lo que si este numero NO es 1 ni -1, sabemos que los
# vectores c1 y c2 no son linealmente independientes.
# Lo mismo vale para c3
# Pero ojo, c3 pudiera no ser proporcional ni a c1 ni a c2 y aun asi ser linealmente dependiente
# de ambos

[[0.17289064]]


In [6]:
n1 = c1/LA.norm(c1)          # dividiendo al vector por su magnitud obtenemos un vector
                                # en la misma direccion pero de magnitud (o "norma") 1.
print(n1)
print(np.dot(np.transpose(n1),n1))  # chequeamos que su norma es 1 multiplicandolo
                                      # escalarmente por si mismo


[[ 0.15941236]
 [-0.98565646]
 [ 0.05539897]]
[[1.]]


In [7]:
# Ya tenemos el vector n1, que es proporcional a c1, pero de magnitud 1.
# Ahora queremos construir el vector n2.
# Recordedmos que n2 es tiene que ser ortonormal con n1, y tal que el span de {n1, n2}
# sea el mismo que el de {c1, c2}
# El mecanismo es: 
# 1) calculamos la componente de c2 paralela a n1, la llamamos P_c2_n1
P_c2_n1 = np.dot(np.transpose(n1),c2)*n1 # la proyeccion del vector c2 sobre el vector n1 es
                                              # (c2.n1) n1 
                                              # c2.n1 es producto escalar de c2 por n1,
                                              # que da la magnitud de la proyeccion de c2 
                                              # sobre sobre n1. Luego, al multiplicarlo por n1,
                                              # el vector resultante apunta en la direccion de n1
                                              # La formula general inluye una division por 
                                              # (n1.n1)
                                              # en este caso no es necesario porque (n1.n1) = 1
print(P_c2_n1)

[[ 0.02201714]
 [-0.13613332]
 [ 0.00765139]]


In [8]:
# 2) Calculamos la componente de c2 perpendicular a n1.
#    A dicha componente la llamamos P_c2_perp_n1 (largo pero explicito... :))
# 2_a) Se calcula simplemente restandole a c2 la componente paralela a n1 que 
#      ya calculamos en el paso 1.
#      Naturalmente, P_c2_perp_n1 tiene que ser perpendicular a n1. 
P_c2_perp_n1 = c2 - P_c2_n1  # la proyeccion del vector c2 perpendicular al vector n1 es
                                  # c2 - P_c2_n1
print(P_c2_perp_n1)
print(np.dot(np.transpose(P_c2_perp_n1),P_c2_n1)) # para chequear la perpendicularidad de
                                                      # P_c2_perp_n1 y P_c2_n1
                                                      # calculamos su producto escalar

[[ 0.46078171]
 [ 0.03874251]
 [-0.63660919]]
[[3.46944695e-18]]


In [9]:
print(c2)                          # Chequamos que c2 = P_c2_n1 + P_c2_perp_n1 de dos maneras
print(P_c2_n1 + P_c2_perp_n1)   # 1- Observando numericamente si coinciden
c2 == P_c2_n1 + P_c2_perp_n1   # 2- Preguntandole a Python con "=="
                                    # (ojo con este ultimo metodo porque puede llegar a dar
                                    # False por errores que las maquinas siempre cometen)

[[ 0.48279885]
 [-0.09739081]
 [-0.6289578 ]]
[[ 0.48279885]
 [-0.09739081]
 [-0.6289578 ]]


array([[ True],
       [ True],
       [ True]])

In [10]:
# 3) Calculamos n2 (que tiene que tener norma 1) simplemente dividiendo a 
#    P_c2_perp_n1 por su norma.
# 3_a) Chequeamos que el conjunto {n1, n2} es ortonormal
#
n2 = P_c2_perp_n1/LA.norm(P_c2_perp_n1) # Generamos n2, que es el vector de norma = 1
print(n2)                                          # perpendicular a n1 
print(np.dot(np.transpose(n2),n2))                # El conjunto {n1, n2} es ortonormal y
print(np.dot(np.transpose(n1),n2))                # genera el subespacio de R3 en el que "viven"
                                                   # c1 y c2

[[ 0.58562233]
 [ 0.04923911]
 [-0.80908714]]
[[1.]]
[[3.46944695e-17]]


In [11]:
# 4) Queremos calcular la componente de c3 perpendicular a n1 y n2 (y por lo tanto,
#    tambien perpendicular a c1 y c2)
# 4_a) Primero calculamos las componentes de c3 paraleleas a n1 y n2.

P_c3_n1 = np.dot(np.transpose(n1),c3)*n1 # Continuamos con el proceso de Gram-Schmidt con
P_c3_n2 = np.dot(np.transpose(n2),c3)*n2 # el vector c3. Ahora estamos calculando las
print(P_c3_n1)                              # proyecciones de c3 sobre n1 y n2
print(P_c3_n2)                          

[[ 0.12186548]
 [-0.75350177]
 [ 0.04235068]]
[[ 0.73174462]
 [ 0.06152506]
 [-1.01096754]]


In [12]:
# 4_b) Calculamos la componente de c3 perpendicular a n1 y n2.
#      A esto lo hacemos simplemente restando de c3 las componentes paralelas a n1 y a n2

P_c3_perp_n1yn2 = c3 - P_c3_n1 - P_c3_n2 # Extraemos la componente de c3 perpendicular al plano
                                         # spanned por n1 y n2
print(P_c3_perp_n1yn2)                     

[[0.10587706]
 [0.02150454]
 [0.07794319]]


In [13]:
# Por como lo construimos, P_c3_perp_n1yn2 tiene que ser ortogonal a n1 y n2
# Comprobemos eso:
print(np.dot(np.transpose(n2),P_c3_perp_n1yn2))               
print(np.dot(np.transpose(n1),P_c3_perp_n1yn2))

[[-2.08166817e-16]]
[[7.2858386e-17]]


In [14]:
# Calculemos entonces el vector proporcional a P_c3_perp_n1yn2 pero normalizado a 1:
n3 = P_c3_perp_n1yn2/LA.norm(P_c3_perp_n1yn2)
print(n3)
# Constatemos que esta normalizado a 1
print(np.dot(np.transpose(n3),n3))
# Esto finaliza nuestro proceso de construccion de una base ortonormal de R3:
# {n1, n2, n3}

[[0.79475417]
 [0.16142137]
 [0.58507175]]
[[1.]]


In [15]:
# Resolver HW 5.14 y 5.15, p. 32, de https://ucema.edu.ar/publicaciones/download/documentos/689.pdf